# *N*-gram Language Models

In this notebook, we'll be building bigram *n*-gram language models from scratch. The first part of building a language model is collecting counts from corpora. We'll do some preprocessing first, by lowercasing everything and add `<s>` (start) and `</s>` (end) symbols at the beginning and end of each sentence. For bigrams, we are using dictionaries of dictionaries with the strings as keys, which is a convenient though not particularly memory efficient way to represent things. We will use the unigram counts later for doing smoothing.

In [1]:
from collections import defaultdict
from collections import Counter


# 给所有句子增加前后符号，并小写化
def convert_sentence(sentence):
    return ["<s>"] + [w.lower() for w in sentence] + ["</s>"]

# 数数句子中单词的数量，并记录bigram和unigram
def get_counts(sentences):
    bigram_counts = defaultdict(Counter)   # 默认value都是Counter, 即{"hello":{'there':2,'!':4},"bye":{"bye":1,"!":3}
    unigram_counts = Counter()
    start_count = 0  # "<s>" counts: need these for bigram probs

    # collect initial unigram statistics
    for sentence in sentences:
        sentence = convert_sentence(sentence)
        for word in sentence[1:]: # from 1, so we don't generate the <s> token，即unigram要跳过符号
            unigram_counts[word] += 1
        start_count += 1 # 每次读过一次句子就统计一个开始符号

    # collect bigram counts
    for sentence in sentences:
        sentence = convert_sentence(sentence)
        # generate a list of bigrams
        bigram_list = zip(sentence[:-1], sentence[1:]) # !!!!!!!! 把一句话尾巴去掉，zip上这句话把头去掉，这样就错位zip，zip在一起的就是（w_i,w_{i+1}）
        # iterate over bigrams
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1 # 这里就是拟了联合概率？ 求P(w_i|w_{i-1})=P(w_i,w_{i-1})/P(w_{i-1})
            
    # 语料库中token的总数
    token_count = float(sum(unigram_counts.values()))
    return unigram_counts, bigram_counts, start_count, token_count

In [2]:
l = zip(["hello","there","nice","to","meet"][:-1],[4,5,6])
print([i for i in l])

[('hello', 4), ('there', 5), ('nice', 6)]


In [6]:
a = defaultdict(Counter)
d = Counter("CCDD")
a.update(d)
print(a)
print(d)

defaultdict(<class 'collections.Counter'>, {'C': 2, 'D': 2})
Counter({'C': 2, 'D': 2})


Once we have counts, we can use them to generate sentences. Here we use [numpy.random.choice](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.choice.html), which allows us to randomly choose from a list based on a corresponding list of probabilities, which we calculate by normalizing the raw counts. We start with &lt;s&gt;, and generate the next word given the bigram counts which begin with &lt;s&gt;, and then use that word to generate the next word, etc. It stops when it generates an &lt;/s&gt;. We return a string with some fixes to make the sentence look a proper sentence.

In [7]:
from numpy.random import choice 

def generate_sentence(bigram_counts):
    
    # 先给句子一个开始符
    current_word = "<s>"
    sentence = [current_word]
    
    #如果当前的不是句子停止符就接着生成
    while current_word != "</s>":
        # get counts for previous word
        prev_word = current_word
        # 拿到了所有第一个词是prev_word的字典，字典里面都是跟在prev_word后面的词，以及他们的频率
        prev_word_counts = bigram_counts[prev_word]
        # obtain bigram probability distribution given the previous word
        bigram_probs = []
        total_counts = float(sum(prev_word_counts.values())) # 拿到了C(w_{i-1})
        for word in prev_word_counts:
            bigram_probs.append(prev_word_counts[word] / total_counts) # 得到 P(w_{i-1},w_i)
        # sample the next word
        # TODO: Randomly choose
        current_word = choice(list(prev_word_counts.keys()), p=bigram_probs)# 因为前面的list是每个current word，后面是每个current word的概率，都是对应的
        sentence.append(current_word)

    # get rid of start and end of sentence tokens
    sentence = " ".join(sentence[1:-1])
    return sentence
        
            

Now let's try out our *n*-gram driven sentence generator on samples from two corpora: the Penn Treebank, and some out-of-copyright English literature from Project Gutenberg:

In [8]:
import nltk
from nltk.corpus import gutenberg, treebank
nltk.download('gutenberg')
nltk.download('treebank')

[nltk_data] Error loading gutenberg: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Hostname mismatch, certificate is not valid for
[nltk_data]     'raw.githubusercontent.com'. (_ssl.c:1123)>
[nltk_data] Error loading treebank: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     Hostname mismatch, certificate is not valid for
[nltk_data]     'raw.githubusercontent.com'. (_ssl.c:1123)>


False

In [9]:
treebank.sents() # 把treebank里面的数据以句子的形式输出

[['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'], ['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.'], ...]

In [10]:
gutenberg_unigrams, gutenberg_bigrams, gutenberg_start_count, gutenberg_token_count = get_counts(gutenberg.sents())
print("Gutenberg")
for i in range(1,6):
    print("Sentence %d" % i)
    print(generate_sentence(gutenberg_bigrams))
    
treebank_unigrams, treebank_bigrams, treebank_start_count, treebank_token_count = get_counts(treebank.sents())
print("Treebank")
for i in range(1,6):
    print("Sentence %d" % i)
    print(generate_sentence(treebank_bigrams))

Gutenberg
Sentence 1
the whole stones he formed a garden ----" she saw her blind and i ' and a most dear jane fairfax ' s very likely , mr .
Sentence 2
the virtue - morrow -- some to pass , and god , and though there but the weight of all that it settings of our pie ; " a proper ; and our tongue , among the jews of her husband : 4 : 20 and turned to the waters , leaping and he that people of a little , camping with a louing , earth .
Sentence 3
36 : 11 : 17 unless , who made an hundred : then dost thou say for my father , why , faint air were their tongue to - trees in their sin .
Sentence 4
19 : 1 : 41 : 40 and not , be as the son of the monarch reign , suddenly by day of the matured and write thee , and emma fancied myself , then i trust in my request , mammy looked about the shower of turnbull , which , he said unto his grandson .
Sentence 5
13 allons !
Treebank
Sentence 1
mr. samnick , ` money manager of the kind of 1987 , staff functions ''
Sentence 2
`` cray 's different items .


Generally, we can see some local coherence but most of these sentences are complete nonsense. Across the two corpora, the sentences are noticeably different, it's very obvious that the model from Project Gutenberg is trained on literature, whereas the Penn Treebank data is financial. For the latter, there are some strange tokens (those starting with \*) we should probably have filtered out.

Using language models to generate sentences is fun but not very useful.（用模型生成语句很有意思但没啥用） A more practical application is the ability to assign a probability to a sentence（给句子赋概率）. In order to do that for anything but toy examples, however, we will need to smooth our models so it doesn't assign a zero probability to the sentence whenever it sees a bigram. Here, we'll test two fairly simple smoothing techniques, add-*k* smoothing and interpolated smoothing. In both cases, we will calculate the log probability, to avoid working with very small numbers. The functions below give the probability for a single word at index i in a sentence.

Notice that interpolation is implemented using 3 probabilities: the bigram, the unigram and a "zerogram" probability. The "zerogram" actually refers to the probability of any word appearing. We need this extra probability in order to account for out-of-vocabulary (OOVs) words, which result in zero probability for both bigrams and unigrams. Estimating the probability of OOVs is a general problem: here we use an heuristic that uses a uniform distribution over all words in the vocabulary (1 / |V|).
在interpolation的时候会涉及到当bigram和unigram都没有的时候，我们会要一个zerogram来避免0概率出现。通常0概率就是OOV的概率，OOV概率又是一个普遍问题，在这里我们用一个启发式方法：用字典里所有词的均匀分布(1 / |V|)来做zeorgram

In [12]:
import math

def get_log_prob_addk(prev_word, word, unigram_counts, bigram_counts, k):
    sm_bigram_counts = bigram_counts[prev_word][word] + k # 当前两个词语的组合数量+k
    sm_unigram_counts = unigram_counts[prev_word] + k*len(unigram_counts) # 前一个单词出现的总次数再加上k倍的token数量
    return math.log(sm_bigram_counts / sm_unigram_counts)

# 获得插值结果，对每一个bigram组合进行差值
def get_log_prob_interp(prev_word, word, unigram_counts, bigram_counts, start_count, token_count, lambdas):
    bigram_lambda = lambdas[0]
    unigram_lambda = lambdas[1]
    zerogram_lambda = 1 - lambdas[0] - lambdas[1]
    
    # start by getting bigram probability
    sm_bigram_counts = bigram_counts[prev_word][word] * bigram_lambda
    if sm_bigram_counts == 0.0:
        # 如果当前数量是0，那bigram插值后就是0，为啥要再写一遍？？
        # 之所以分开讨论应该是避免prev_word也不存在的情况，这样求概率的时候分母就会为0
        interp_bigram_counts = 0
    else:
        # 如果不是0，先判断前一个词是不是开头，是的话就直接用句子数量
        if prev_word == "<s>":
            u_counts = start_count
        else:
            # 如果不是开头，通过unigram对这个词的统计，得到前一个单词的数量
            u_counts = unigram_counts[prev_word]
        # 这里计算的是插值后的两个词组数量，除去prev_word，就是插值后的概率
        interp_bigram_counts = sm_bigram_counts / float(u_counts)
        
    # unigram probability
    interp_unigram_counts = (unigram_counts[word] / token_count) * unigram_lambda
    
    # "zerogram" probability: this is to account for out-of-vocabulary words
    # this is just 1 / |V|
    vocab_size = len(unigram_counts)
    interp_zerogram_counts = (1 / float(vocab_size)) * zerogram_lambda
    
    return math.log(interp_bigram_counts + interp_unigram_counts + interp_zerogram_counts)


Extending this to calculate the probability of an entire sentence is trivial.

In [13]:
def get_sent_log_prob_addk(sentence, unigram_counts, bigram_counts, start_count, token_count, k):
    sentence = convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    
    # 给一句话，把这句话里面的概率都求一遍，因为是log所以用加法
    return sum([get_log_prob_addk(prev_word, 
                                  word, 
                                  unigram_counts, 
                                  bigram_counts, 
                                  k) for prev_word, word in bigram_list])

def get_sent_log_prob_interp(sentence, unigram_counts, bigram_counts, start_count, token_count, lambdas):
    sentence = convert_sentence(sentence)
    bigram_list = zip(sentence[:-1], sentence[1:])
    return sum([get_log_prob_interp(prev_word, 
                                    word, 
                                    unigram_counts, 
                                    bigram_counts,
                                    start_count,
                                    token_count, 
                                    lambdas) for prev_word, word in bigram_list])
    
sentence = "revenue increased last quarter .".split()
print(get_sent_log_prob_addk(sentence, gutenberg_unigrams, gutenberg_bigrams, gutenberg_start_count,
                             gutenberg_token_count, 0.05))
print(get_sent_log_prob_interp(sentence, 
                               gutenberg_unigrams, 
                               gutenberg_bigrams, 
                               gutenberg_start_count, 
                               gutenberg_token_count, 
                               (0.8, 0.19)))
print(get_sent_log_prob_addk(sentence, treebank_unigrams, treebank_bigrams, treebank_start_count,
                             treebank_token_count, 0.05))
print(get_sent_log_prob_interp(sentence, 
                               treebank_unigrams, 
                               treebank_bigrams, 
                               treebank_start_count, 
                               treebank_token_count, 
                               (0.8, 0.19)))

-48.460406447015146
-49.538820071127226
-39.776378681452364
-39.245480234555636


In [17]:
math.e**(-39.245480234555636)

9.034507744082557e-18

The output for our sample sentence looks reasonable, in particular using the Treebank model results in a noticeably higher probability, which is what we'd expect given the input sentence. The differences in probability between the different smoothing techniques is more modest (though keep in mind this is a logrithmic scale). Now, let's use perplexity to evaluate different smoothing techniques at the level of the corpus. For this, we'll use the Brown corpus again, dividing it up randomly into a training set and a test set based on an 80/20 split.

In [14]:
from nltk.corpus import brown
from random import shuffle
nltk.download('brown')

sents = list(brown.sents())
shuffle(sents)
cutoff = int(0.8*len(sents))
training_set = sents[:cutoff]
test_set = [[word.lower() for word in sent] for sent in sents[cutoff:]]

brown_unigrams, brown_bigrams, brown_start_count, brown_token_count = get_counts(training_set)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\IvanDeng\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


Since our probabilities are in log space, we will calculate perplexity in log space as well, then take the exponential at the end

$PP(W) = \sqrt[m]{\frac{1}{P(W)}}$

$\log{PP(W)} = -\frac{1}{m} \log{P(W)}$

https://www.zhihu.com/question/58482430 \
困惑度：语言模型的效果好坏的常用评价指标是困惑度(perplexity),**句子概率越大，语言模型越好，迷惑度越小。**\
log perplexity 可以看作真实分布与预测分布之间的交叉熵 Cross Entropy, 交叉熵描述了两个概率分布之间的一种距离

In [15]:
def calculate_perplexity(sentences, unigram_counts, bigram_counts, start_count, 
                         token_count, smoothing_function, parameter):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 1 # have to consider the end token
        total_log_prob += smoothing_function(sentence,
                                             unigram_counts,
                                             bigram_counts,
                                             start_count,
                                             token_count,
                                             parameter)
    # 句子的log概率除去当前句子长度
    return math.exp(-total_log_prob / test_token_count)

                

Let's see how our two smoothing techniques do with a range of possible parameter values 

In [16]:
print("add k")
for k in [0.0001,0.001,0.01, 0.05,0.2,1.0]:
    print(k)
    print(calculate_perplexity(test_set,
                               brown_unigrams,
                               brown_bigrams,
                               brown_start_count,
                               brown_token_count,
                               get_sent_log_prob_addk,
                               k))
print("interpolation")
for bigram_lambda in [0.98,0.95,0.75,0.5,0.25,0.001]:
    unigram_lambda = 0.99 - bigram_lambda
    lambdas = (bigram_lambda, unigram_lambda)
    print(lambdas) 
    print(calculate_perplexity(test_set, 
                               brown_unigrams, 
                               brown_bigrams,
                               brown_start_count,
                               brown_token_count, 
                               get_sent_log_prob_interp, 
                               lambdas))

add k
0.0001
742.8586619814255
0.001
604.3736239487934
0.01
712.7983394857446
0.05
1031.5331251867012
0.2
1682.9713850153091
1.0
3497.166991369912
interpolation
(0.98, 0.010000000000000009)
756.4687282790394
(0.95, 0.040000000000000036)
580.4687517527991
(0.75, 0.24)
422.2158036564698
(0.5, 0.49)
419.91144136656897
(0.25, 0.74)
494.7855394287438
(0.001, 0.989)
979.3125045486988


Our results indicate that, with regards to perplexity, interpolation is generally better than add k. Very low (though not too low) k is preferred for add k, wheres our best lambdas is in the middle of the range, though apparently with a small preference for more weight on the bigram probability, which makes sense.

From the basis given here, you can try playing around with some of the other corpora in NLTK and see if you get similar results. You could also implement a trigram model, or another kind of smoothing, to see if you can get better perplexity scores.